# Bulk Forecast

Geração de previsões para multiplas categorias

In [1]:
import pandas as pd
import numpy as np
import json

from fbprophet import Prophet

in_data = pd.read_csv('geral.csv', index_col=0)
categories = json.load(open('subcategorias.json'))
out_data = pd.DataFrame(np.zeros((len(categories.keys()), 11)), columns=['Mes', 'Ano', 'Categoria', 'Produto', 'Compra sugerida', 'Venda prevista', 'Venda anterior', 'Estoque atual', 'Estoque anterior', 'Valor venda', 'Valor anterior'])

In [103]:
def get_forecast_figure(train, periods):
    ph = Prophet(daily_seasonality=False)
    ph.fit(pd.DataFrame({'ds': train.index, 'y':train.values}))

    future = ph.make_future_dataframe(periods=periods, freq='D')
    forecast = ph.predict(future)
    
    forecast.tail()
    
    return np.round(forecast['yhat'].sum())

## Geração da tabela de fatos
Adicionar compra sugerida, venda prevista, estoque atual, estoque anterior, valor venda, valor anterior.

In [19]:
i = 0
for key in categories.keys():
    in_data[key] = in_data[categories[key]].sum(axis=1)
    row = pd.Series([3, 2021, 
                     key, key, 
                     np.random.randint(0, 99), # Compra sugerida
                     np.random.randint(0, 999), # Venda prevista
                     in_data.loc['2021-03-01':'2021-03-31', key].sum(), # Venda anterior
                     np.random.randint(0, 99), # Estoque atual
                     np.random.randint(0, 99), # Estoque anterior
                     np.random.randint(0, 999999), # Valor venda
                     np.random.randint(0, 999999)]) # Valor anterior
    out_data.iloc[i] = row
    i+= 1

In [20]:
in_data.to_csv('geral.csv') # Salvar a serie temporal de vendas com a adição das categorias

In [3]:
out_data.to_csv('fatos.csv')

In [4]:
in_data.index = pd.to_datetime(in_data.index)

In [6]:
category_products = categories['ELETRO LINHA BRANCA']
filtered_data = in_data[category_products]
filtered_data.index = pd.to_datetime(filtered_data.index)
filtered_data = filtered_data.loc[(in_data.index.month == 2) & (in_data.index.year == 2021)]

if(len(category_products) < 5): # Em algumas categorias o número de produtos pertencentes é menor que 5
    top_5 = filtered_data.sum().sort_values(ascending=False)
else:
    top_5 = filtered_data.sum().sort_values(ascending=False)[:5]

In [13]:
a = {product: sales for (product, sales) in zip(top_5.index, top_5)}

In [21]:
df = pd.DataFrame({'produto': top_5.index, 'qtd vendida': top_5.values}, index=range(1, 6))

In [22]:
df.to_dict('records')

[{'produto': 'FRITADEIRA AIR FRYER AF-30 I FAMILY IV 3,5L',
  'qtd vendida': 1306},
 {'produto': 'REFRIGERADOR ROC 35 BR', 'qtd vendida': 879},
 {'produto': 'LAVADORA PAOLA 10KG PRETA', 'qtd vendida': 744},
 {'produto': 'FOGAO 4BC. AGATA 4062 BR', 'qtd vendida': 556},
 {'produto': 'LAVADORA PAOLA 10KG BR/LILAS', 'qtd vendida': 496}]